In [1]:
import MeCab as mc
import re
import pandas as pd
import numpy as np
import urllib.request
import unicodedata

In [3]:
#retweet dataの読み込み
retweet_input = pd.read_csv(filepath_or_buffer="Datas/t_retweet.csv", encoding="utf_8", sep=",")
print(len(retweet_input))

#tweet dataの読み込み
tweet_input = pd.read_csv(filepath_or_buffer="Datas/t_tweet_data.csv", encoding="utf_8", sep=",")
print(len(tweet_input))

#tweet dataの読み込み
userID_input = pd.read_csv(filepath_or_buffer="Datas/t_tweet_ids.csv", encoding="utf_8", sep=",")
print(len(userID_input))

#retweetしたデータとしなかったデータに分割
retweet = retweet_input.query('retweet == 1')
n_retweet = retweet_input.query('retweet == 0')

#retweetしたデータ
r_data = pd.merge(retweet, tweet_input, how="inner", on="tweetID")
#retweetしなかったデータ
nr_data = pd.merge(n_retweet, tweet_input, how="inner", on="tweetID")

#余分に別れたカラムの結合
r_data['tweet2'] = r_data[['tweet', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']].apply(lambda x: '{}_{}_{}'.format(x[0], x[1], x[2],x[3],x[4]), axis=1)
nr_data['tweet2'] = nr_data[['tweet', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']].apply(lambda x: '{}_{}_{}'.format(x[0], x[1], x[2],x[3],x[4]), axis=1)

#必要なカラム以外は取り除く
r_data.drop(['date_y','tweet','Unnamed: 3', 'Unnamed: 4','Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)
nr_data.drop(['date_y','tweet','Unnamed: 3', 'Unnamed: 4','Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)

#不要な文字の削除
r_data["tweet2"] = r_data["tweet2"].str.replace('\n', ' ')
r_data["tweet2"] = r_data["tweet2"].str.replace('_nan', ' ')
nr_data["tweet2"] =nr_data["tweet2"].str.replace('\n', ' ')
nr_data["tweet2"] = nr_data["tweet2"].str.replace('_nan', ' ')

print(r_data.head(3))
print(nr_data.head(3))

48298
48769
64925
        tweetID  retweet               date_x  \
0  7.113431e+17        1  2016-03-20 00:00:00   
1  7.124711e+17        1  2016-03-23 00:00:00   
2  7.223425e+17        1  2016-04-19 00:00:00   

                                              tweet2  
0                         本日も遠野風の丘軒下市、よろしくお願いします。     
1  【遠野山ぶどうワイン2015】本日より発売開始です！！駅前売店でも入荷しております♪数量限定...  
2  骨寺荘園室より 【世界遺産 拡張資産 巡回バス】のお知らせ  ４月23日（土）に世界遺産拡張...  
        tweetID  retweet               date_x  \
0  7.111280e+17        0  2016-03-19 00:00:00   
1  7.113160e+17        0  2016-03-20 00:00:00   
2  7.113729e+17        0  2016-03-20 00:00:00   

                                              tweet2  
0  きょうは来てくれてありがとっ♡ お買い物してくれてありがとっ♡ 21日は陸前高田にねば〜るく...  
1  みんなゆめ☆も〜にん☆彡  かさ上げに 思いはひとつ 笑顔かなっ♪            ゆ...  
2  【遠野風の丘軒下市】ラム肉まんが通常220円のところ200円、2つで300円で販売しています...  


In [5]:
#データをランダムに5000件ずつ抽出
ran_r_data = r_data.sample(n=5000)
ran_nr_data = nr_data.sample(n=5000)
#データの結合
data = ran_r_data.append(ran_nr_data)
#dateでソート
data_s = data.sort_values('date_x')
print(data_s.head(10))

         tweetID  retweet               date_x  \
0   7.113431e+17        1  2016-03-20 00:00:00   
11  7.117613e+17        0  2016-03-21 00:00:00   
16  7.117752e+17        0  2016-03-21 00:00:00   
17  7.117819e+17        0  2016-03-21 00:00:00   
32  7.122291e+17        0  2016-03-22 00:00:00   
31  7.122290e+17        0  2016-03-22 00:00:00   
40  7.124736e+17        0  2016-03-23 00:00:00   
1   7.124711e+17        1  2016-03-23 00:00:00   
64  7.132281e+17        0  2016-03-25 00:00:00   
63  7.132280e+17        0  2016-03-25 00:00:00   

                                               tweet2  
0                          本日も遠野風の丘軒下市、よろしくお願いします。     
11                  今日のお昼。喫茶のんのんのカッパフェー桜バージョン(^-^)     
16                                     よいしょっよいしょっ♡     
17                                       みんなありがとっ♡     
32                                    遠野職業訓練校修了式       
31                                      鱒沢小学校卒業式       
40                                  青笹町地域づくり推進大会       
1

In [63]:
#User pickup
u_data = []
count= 1

for i in data_s["tweetID"]:
    ext = userID_input[userID_input["tweetID"] == i]
    #u_data.append(ext)
    if count == 1:
        df_uID = ext
        count +=1
    else:
        df_uID = pd.concat([df_uID, ext])

In [66]:
#userID
ids = df_uID.drop(['tweetID'], axis=1)
ids.to_csv("user.txt", header=False, index=False, sep=",")

In [15]:
#テキストデータのみ取り出し、ファイルに書き出す
text = data_s.drop(['tweetID', 'retweet', 'date_x'], axis=1)
text.to_csv("tweet.txt",header=False, index=False, sep=",")
#retweetのラベルのみ取り出す
tweet_labe = data_s.drop(['tweetID', 'date_x', 'tweet2'], axis=1)
tweet_labe.to_csv("label.txt", header=False, index=False, sep=",")

In [16]:
#テキストのクリーニング
def clean_text(text_string):
    text_string = re.sub(r'([^\s\w])+', '', text_string)
    text_string = re.sub(r'[0-9]', '', text_string)
    text_string = re.sub(r'[０-９]', '', text_string)
    text_string = " ".join(text_string.split())
    text_string = text_string.lower()
    return(text_string)

#slothlibのストップワードの取得
def get_stopword():
    slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    slothlib_file = urllib.request.urlopen(slothlib_path)
    slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
    slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
    return slothlib_stopwords

#ストップワードの追加
def add_stopword(slothlib_stopwords):
    s = open("Datas/stopwords.txt", "r", encoding="utf-8")
    stop = s.readlines()
    s.close()
    stop = [a.strip() for a in stop]
    slothlib_stopwords += stop
    return slothlib_stopwords

#MeCabによる形態素解析
def extractter(text, flag):
    text = unicodedata.normalize("NFKC", text)
    tagger = mc.Tagger(r'-Ochasen -d G:\neologd')
    tagger.parse('')
    node = tagger.parseToNode(text)
    key= []
    word = []
    keyword =[]
    while node:
        nes = node.feature.split(",")
        wor = node.surface
        if wor != "":
            nes = node.feature.split(",")
            if nes[0] == u"名詞":
                if nes[6] == u"*":
                     keyword.append(wor)
                else:
                    keyword.append(nes[6])
        node = node.next
        if node is None:
            break
    return keyword

#ストップワードの除去
def except_stopwords(text, stopwords):
    for i in text:
        for j in stopwords:
            if i == j:
                text.remove(j)
    return text

In [17]:
#メイン
#ツイートのテキスト読み込み
test = open("tweet.txt", "r", encoding="utf-8")
lines = test.readlines()
test.close()
print(len(lines))

#クリーニング及び形態素解析
text_data = [clean_text(x) for x in lines]
M_extract_text = [extractter(y, 0) for y in text_data]
stopwords = get_stopword()
ex_stopwords = add_stopword(stopwords)
M_result = [except_stopwords(a, ex_stopwords) for a in M_extract_text]
M_result = [' '.join(d) for d in M_result]

10000


In [18]:
#書き出し1
g = open("extract_tweet.txt", "w", encoding='utf-8')
for i in M_result:
    g.write(i)
    g.write('\n')
g.close()